# Implementation of The Decision Tree Classifier

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import math
import pydotplus
from collections import deque

In [2]:
class TreeNode:
    # This class will be used to create objects that will act as nodes of the decision tree
    def __init__(self, data, output):
        # Data => The feature upon which the split has taken place.
        #         If Node is a leaf Node, data = None
        self.data = data
        # Children of a node is stored as a dicitonary in which:
        # Key = The value at which the feature is split
        # Value = The child tree node
        self.children = {}
        # Output represents the current majority class
        self.output = output
        # Index => to distinguish to different nodes in the tree
        self.index = -1
        
    # Function to add children of node to the dictionary of child nodes
    def add_children(self, feature_value, obj):
        self.children[feature_value] = obj

In [20]:
class DecisionTreeClassifier:
    # This class will consist of all the required functions for the working of the decision tree classifier
    def __init__(self):
        # Initializing the root node of the tree with None, intended to be a private variable.
        self.__root = None
    
    def __count_unqiue(self, Y):
        # Returns a dictionary that consists of possible ouptut classes and their frequencies
        temp = {}
        for i in Y:
            if i not in temp:
                temp[i] = 1
            else:
                temp[i] = temp[i] + 1
        return temp
    
    def __entropy(self, Y):
        # This function will calculate the entropy using the formula as we know.
        
        # Getting frequencies of all classes Y
        freqs = self.__count_unqiue(Y)
        total = len(Y)
        # Init entropy with 0
        entropy = 0
        for i in freqs:
            # Probability
            p = freqs[i]/total
            entropy += (-p)*math.log2(p)
        return entropy
            
    def __gain_ratio(self, X, Y, selected_feature):
        # This function computes the gain ratio of the feature selected for splitting.
        entropy_before_split = self.__entropy(Y)
        # Entropy after splitting on current feature
        entropy_after_split = 0
        split_info = 0
        # different values of the selected feature, as we need to determine what value to split on for
        # the selected feature
        values = set(X[:, selected_feature])
        # Forming a data frame
        df = pd.DataFrame(X)
        # Adding Y to the last column
        df[df.shape[1]] = Y
        initial_size = df.shape[0]
        # Iterate on each value of the selected feature
        for i in values:
            # Select rows with value equal to current value
            temp_df = df[df[selected_feature] == i]
            # Size of the new data frame that has rows with only current value
            current_size = temp_df.shape[0]
            entropy_after_split += (current_size/initial_size)*self.__entropy(temp_df[temp_df.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)
            
        # Hanlde case where split_info = 0, to avoid division by zero.
        if split_info == 0:
            return math.inf
        
        information_gain = entropy_before_split - entropy_after_split
        gain_ratio = information_gain / split_info
        return gain_ratio
    
    def __decision_tree(self, X, Y, features, level, metric, classes):
        # Metric will be gain ratio, in future may be implement gini index as well
        
        # This function will return the root of the decision tree after fitting the data.
        # A feature is split upon only once.
        # Level represents the depth of the tree.
        # Classes is the set of possible labels/outputs for the dataset.
        
        
        # Base Case 1
        # Node is a leaf node, that is only 1 possible value of Y.
        if len(set(Y)) == 1:
            print("Level ", level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of ", i, " = ", len(Y))
                else:
                    print("Count of ", i, " = 0")
            if metric == "gain_ratio":
                # Entropy is 0.0 since pure node
                print("Current Entropy is 0.0")
            
            print("Reached Leaf Node")
            print()
            # Form the node.
            return TreeNode(None, output)
        
        # Base Case 2
        # All features exhausted, feature to split upon will be deciding using majority.
        if len(features) == 0:
            print("Level ", level)
            # Get the unique classes present and their frequencies
            freqs = self.__count_unqiue(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freqs:
                    print("Count of ", i, " = 0")
                else:
                    if freqs[i] > max_count:
                        # Update max and output
                        output = i
                        max_count = freqs[i]
                    print("Count of ", i, " = ", freqs[i])
                if metric == "gain_ratio":
                    print("Current Entropy is ", self.__entropy(Y))
                    
            print("Reached Leaf Node.")
            print()
            return TreeNode(None, output)
        
        # Finding best feature to split upon.
        # Initializing maximum gain with -ve infinite.
        max_gain = -math.inf
        split_feature = None
        for f in features:
            if metric == "gain_ratio":
                current_gain = self.__gain_ratio(X, Y, f)
            if current_gain > max_gain:
                max_gain = current_gain
                split_feature = f
        print("Level ", level)
        # Get count of different classes in Y
        freqs = self.__count_unqiue(Y)
        output = None
        max_count = -math.inf
        
        for i in classes:
            if i not in Y:
                print("Count of ", i, " = 0")
            else:
                if freqs[i] > max_count:
                    output = i
                    max_count = freqs[i]
                print("Count of ", i, " = ", freqs[i])
        
        if metric == "gain_ratio":
            print("Current Entropy is ", self.__entropy(Y))
            print("Splitting on feature  X[", split_feature, "] with gain ratio ", max_gain, sep="")
            print()
        
        # Unique values of the feature selected
        unique_vals_ft = set(X[:, split_feature])
        
        # Create dataframe
        df = pd.DataFrame(X)
        # Add labels
        df[df.shape[1]] = Y
        
        # Make a tree node
        current_node = TreeNode(split_feature, output)
        
        # Remove the feature on which we have split from list of features
        # Inorder to avoid splitting on this feature any further
        index_ft = features.index(split_feature)
        features.remove(split_feature)
        
        # Iterate on different vals the feature can take
        for i in unique_vals_ft:
            # Create dataframe with only this value of the current feature
            temp_df = df[df[split_feature] == i]
            # Recursively call on different values of the selected feature
            node = self.__decision_tree(temp_df.iloc[:, 0:temp_df.shape[1]-1].values, temp_df.iloc[:, temp_df.shape[1]-1].values,
                                       features, level+1,metric,classes)
            # Add this node as child of current node
            current_node.add_children(i, node)
        
        # Add the removed feature back to list
        features.insert(index_ft, split_feature)
        return current_node
    
    def fit(self, X, Y, metric="gain_ratio"):
        # Fits the training data to the classifier
        
        # Initializing parameters
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        
        self.__root = self.__decision_tree(X, Y, features, level, metric, classes)
    
    def __predictSinglePoint(self, data, node):
        # Predicts the class for a single data point
        
        # If we reach a leaf node.
        if len(node.children) == 0:
            return node.output
        
        # Value on which split was made
        split_val = data[node.data]
        if split_val not in node.children:
            return node.output
        
        # Recursively call splits
        return self.__predictSinglePoint(data, node.children[split_val])
    
    def predict(self, X):
        # This function will return the Y_pred array
        Y_pred = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            # Predict for single data pt
            Y_pred[i] = self.__predictSinglePoint(X[i], self.__root)
        return Y_pred
    
    def score(self, X, Y):
        # Score using mean accuracy
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count += 1
        return count / len(Y_pred)
    
    # Function to export tree pdf
    def export_tree(self, filename=None):
        # Returns dot data about the tree
        # If filename specified will also store a pdf in current directory with visual representation of tree
        
        dot_data = '''digraph Tree {node [shape=box] ;'''
        
        queue = deque()
        r = self.__root
        queue.append(r)
        count = 0
        if r.index == -1:
            r.index = count
        
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.data,r.output) 
        
        # Performing Level Order Traversal
        while len(queue) != 0:
            node = queue.popleft()
            for i in node.children:
                count += 1
                # Assign index to node
                if node.children[i].index == -1:
                    node.children[i].index = count
                # Creating a child node
                dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(node.children[i].index,node.children[i].data,node.children[i].output) 
                # Connecting parent node with child
                dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
                # Adding child node to queue
                queue.append(node.children[i])
        dot_data = dot_data + "\n}"
                
        if filename != None:
            graph = pydotplus.graph_from_dot_data(dot_data)
            graph.write_pdf(filename)
        return dot_data

In [21]:
# 2 Input OR GATE 
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0, 1, 1, 1])
clf = DecisionTreeClassifier()
clf.fit(X, Y)
Y_pred = clf.predict(X)
print("Predictions ", Y_pred)
print()
print("Score = ", clf.score(X, Y))

Level  0
Count of  0  =  1
Count of  1  =  3
Current Entropy is  0.8112781244591328
Splitting on feature  X[0] with gain ratio 0.31127812445913283

Level  1
Count of  0  =  1
Count of  1  =  1
Current Entropy is  1.0
Splitting on feature  X[1] with gain ratio 1.0

Level  2
Count of  0  =  1
Count of  1  = 0
Current Entropy is 0.0
Reached Leaf Node

Level  2
Count of  0  = 0
Count of  1  =  1
Current Entropy is 0.0
Reached Leaf Node

Level  1
Count of  0  = 0
Count of  1  =  2
Current Entropy is 0.0
Reached Leaf Node

Predictions  [0 1 1 1]

Score =  1.0


## Applying On Iris Dataset

In [22]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [23]:
# Iris dataset loading
iris = datasets.load_iris()
X = iris.data
Y = iris.target
X.shape, Y.shape

((150, 4), (150,))

In [24]:
# Making a dataframe
iris_df = pd.DataFrame(X)
iris_df[iris_df.shape[1]] = Y
columns = iris.feature_names
columns.append('Output')
iris_df.columns = columns
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Output
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


In [25]:
# Splitting into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

### Using our own classifier

In [26]:
clf_own = DecisionTreeClassifier()

In [27]:
clf_own

In [28]:
# Fitting data
clf_own.fit(x_train, y_train)

Level  0
Count of  0  =  37
Count of  1  =  34
Count of  2  =  41
Current Entropy is  1.5807197138422102
Splitting on feature  X[3] with gain ratio 0.35471104425168914

Level  1
Count of  0  =  5
Count of  1  = 0
Count of  2  = 0
Current Entropy is 0.0
Reached Leaf Node

Level  1
Count of  0  = 0
Count of  1  =  6
Count of  2  =  2
Current Entropy is  0.8112781244591328
Splitting on feature  X[2] with gain ratio 0.2950102270760483

Level  2
Count of  0  = 0
Count of  1  =  1
Count of  2  = 0
Current Entropy is 0.0
Reached Leaf Node

Level  2
Count of  0  = 0
Count of  1  =  1
Count of  2  = 0
Current Entropy is 0.0
Reached Leaf Node

Level  2
Count of  0  = 0
Count of  1  =  1
Count of  2  = 0
Current Entropy is 0.0
Reached Leaf Node

Level  2
Count of  0  = 0
Count of  1  = 0
Count of  2  =  1
Current Entropy is 0.0
Reached Leaf Node

Level  2
Count of  0  = 0
Count of  1  =  2
Count of  2  = 0
Current Entropy is 0.0
Reached Leaf Node

Level  2
Count of  0  = 0
Count of  1  =  1
Count

In [29]:
# Predicting
Y_pred = clf_own.predict(x_test)
Y_pred

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       2, 0, 1, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])

In [30]:
# Score
print('Score (Mean accuracy) of our own implementation is: ', clf_own.score(x_test, y_test))

Score (Mean accuracy) of our own implementation is:  0.9473684210526315


In [31]:
# Exporting Tree PDF
print("DOT DATA :-",clf_own.export_tree(filename="IRIS_TREE.pdf"))

DOT DATA :- digraph Tree {node [shape=box] ;
0 [label="Feature to split upon : X[3]\nOutput at this node : 2" ];
1 [label="Feature to split upon : X[None]\nOutput at this node : 0" ];
0 -> 1 [ headlabel="Feature value = 0.3"]; 
2 [label="Feature to split upon : X[2]\nOutput at this node : 1" ];
0 -> 2 [ headlabel="Feature value = 1.5"]; 
3 [label="Feature to split upon : X[None]\nOutput at this node : 2" ];
0 -> 3 [ headlabel="Feature value = 2.1"]; 
4 [label="Feature to split upon : X[None]\nOutput at this node : 1" ];
0 -> 4 [ headlabel="Feature value = 1.3"]; 
5 [label="Feature to split upon : X[1]\nOutput at this node : 2" ];
0 -> 5 [ headlabel="Feature value = 1.8"]; 
6 [label="Feature to split upon : X[None]\nOutput at this node : 2" ];
0 -> 6 [ headlabel="Feature value = 2.3"]; 
7 [label="Feature to split upon : X[None]\nOutput at this node : 0" ];
0 -> 7 [ headlabel="Feature value = 0.2"]; 
8 [label="Feature to split upon : X[2]\nOutput at this node : 1" ];
0 -> 8 [ headlabel="

## Using Sklearn's inbuilt decision tree classifier and comparing

In [32]:
import sklearn.tree

In [33]:
clf_skl = sklearn.tree.DecisionTreeClassifier(criterion='entropy')
clf_skl

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [34]:
# Fitting
clf_skl.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [18]:
# predicitng
Y_pred = clf_skl.predict(x_test)
Y_pred

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])

In [35]:
print('Score of SKLEARN inbuilt DecisionTreeClassifier on Iris Dataset is: ', clf_skl.score(x_test, y_test))

Score of SKLEARN inbuilt DecisionTreeClassifier on Iris Dataset is:  0.9736842105263158
